In [8]:
import numpy as np 
from scipy.stats import multivariate_normal
import itertools

# The class which defines the graph, transitions, and solver

In [9]:
# class graph_mdp():

#     def __init__(self,n_states,n_actions,rewards,init_state,init_P,init_v):
#         self.n_states = n_states                
#         self.n_actions = n_actions
#         self.r = rewards 
#         self.state = init_state
#         self.P = init_P
#         self.Q = init_Q
        
#     def transition_probs(self,action):
        
#         # get the post-interv. essential graphs
#         post_int_graphs = post_intervention_graphs(action)
        
#         # calculate the BIC for each one 
#         bic_scores = graphBIC(post_int_graphs)
        
#         # transform BIC scores to probabilities
#         self.P = ... 
        
#     def post_intervention_graphs(action):
#         """
#             Given current graph (self.state) 
#             and intervention on that graph (action)
#             return the set of post intervention
#             graphs (next states)
#         """
        
#         # 
        
#         return graphs
    
#     def normal_LL(x,y,beta,sigma):
#         mu = beta.dot(x.T)
#         ll = norm(loc=mu, scale=sigma).logpdf(y).sum()
#         return ll
                
    
#     def graphBIC(self,post_int_graphs):
        
#         def gBIC(g):
            
        
#         # calculate BIC score for every graph 
#         bic_scores=[]
#         for g in post_int_graphs:
#             bic_scores.append()
            
        
#         return bic_scores
    
#     def solve(self):
#         """
#             Given a max number of experiments
#             return the optimal policy which is 
#             the sequence of experiment on 
#             how to go from observed data to the 
#             causal graph
#         """
                
        
#         return causal_graph


    

## Below this cell is an experimentation section. Go to the last cell to run the experiment.

In [10]:
# example of adjacency matrix for 3 variables: x1, x2, x3


adj_matrices = {
    
    # A1 describes a graph where the edges are x1 -> x2 -> x3 
    "A1" : [[0,0,0],
            [1,0,0],
            [0,1,0]],

    # A2 describes a graph where the edges are x2 -> x1 -> x3 
    "A2" : [[0,1,0],
            [0,0,0],
            [1,0,0]],

    # A3 : x3,x2 -> x1
    "A3" : [[0,1,1],
            [0,0,0],
            [0,0,0]],

    # A4 : x2, x1, x3 (independent)
    "A4": [[0,0,0],
            [0,0,0],
            [0,0,0]],


    # A5 : x1 -> x2, x1,x2 -> x3
    "A5" : [[0,0,0],
            [1,0,0],
            [1,1,0]],

    # AC describes a fully connected, undirected graph (not causal)
#     "AC": [[0,1,1],
#             [1,0,1],
#             [1,1,0]]
}

In [11]:
def generate_dataset(N,true_g,sigma):
    
    if true_g == "A1":
        x1 = np.random.normal(loc=0,scale=sigma,size=N)
        x2 = x1 + np.random.normal(loc=0,scale=sigma,size=N)
        x3 = x2 + np.random.normal(loc=0,scale=sigma,size=N)
        
    elif true_g == "A2":
        x2 = np.random.normal(loc=0,scale=sigma,size=N)
        x1 = x2 + np.random.normal(loc=0,scale=sigma,size=N)
        x3 = x1 + np.random.normal(loc=0,scale=sigma,size=N)
      
    elif true_g == "A3":
        x2 = np.random.normal(loc=0,scale=sigma,size=N)
        x3 = np.random.normal(loc=0,scale=sigma,size=N)        
        x1 = x2 + x3 + np.random.normal(loc=0,scale=sigma,size=N)
    
    elif true_g == "A4":
        x1 = np.random.normal(loc=0,scale=sigma,size=N)
        x2 = np.random.normal(loc=0,scale=sigma,size=N)
        x3 = np.random.normal(loc=0,scale=sigma,size=N)
        
    elif true_g == "A5":
        x1 = np.random.normal(loc=0,scale=sigma,size=N)
        x2 = x1 + np.random.normal(loc=0,scale=sigma,size=N)
        x3 = x1 + x2 + np.random.normal(loc=0,scale=sigma,size=N)
    else:
        print(f"Graph {true_g} not found.")
        
    x = np.c_[x1,x2,x3]
    y = x.T
    return x,y

In [12]:
def log_lik(beta, y, x):

    mu = beta.dot(x.T)
    neg_llik = -1*norm(loc=mu, scale=1).logpdf(y).sum()

    return neg_llik

In [26]:
from scipy.special import softmax

# Gym-like wrapper
class LabeledMDP():
    def __init__(self, true_graph_name, max_time = 200):
        self.true_graph = np.asarray(adj_matrices[true_graph_name])
        self.true_graph_name = true_graph_name
        self.n_nodes = np.asarray(self.true_graph).shape[0]
        self.max_time = max_time

        # state space
        self.curr_graph = self.get_reset_graph()
        self.t = 0 # keep track of the timestep
        self.n_experiments = 0 # experiments run so far
        self.intervened_nodes = {} # keep track of which nodes have been intervened on

        # action space
        # for each node, there are experiments with three different choices n: {10, 100, 1000}
        self.experiment_sizes = [10, 100, 1000]
        self.experiment_actions = np.array(list(itertools.product(np.arange(self.n_nodes), self.experiment_sizes))).flatten()
        self.n_actions = len(self.experiment_actions) # add one for the stop action

    def get_reset_graph(self):
        return np.ones((self.n_nodes, self.n_nodes)) # the totally connected graph

    def reset(self):
        self.t = 0
        self.curr_graph = self.get_reset_graph()
        self.intervened_nodes = {}
        self.n_experiments = 0
        return self.create_state()

    def create_state(self):
        frac = len(self.intervened_nodes.keys()) / self.n_nodes
        s = np.concatenate((self.curr_graph.flatten(), [self.n_experiments, frac]))
        assert s.ndim == 1, "Not 1D feature vector"
        return s

    def correct_edges_frac(self):
        # compare correct and incorrect edges
        # use for shaped reward if desired
        n_correct_edges = np.sum(self.curr_graph == self.true_graph)
        return n_correct_edges / np.prod(self.curr_graph.shape)

    def transition_graph(self, node, n_samples):
        # define candidate graphs the agent considers
        # for now, just set it to be all possible graphs
        candidate_graphs = list(adj_matrices.keys())

        # std dev of generated data
        sigma = 1.

        x,Y = generate_dataset(n,self.true_graph_name,sigma)
        ll=[]
        bic_scores = []
        for g_name in candidate_graphs:
            G = np.array(adj_matrices[g_name])
            graph_ll=0
            for var in range(self.n_nodes):
                data = Y[var]
                predictors = np.einsum('ij,ij->ij',np.tile(G[var],(n,1)),x)
                opt_res = optimize.minimize(fun = log_lik, 
                                        x0 = [0, 0, 0], 
                                        args = (data, predictors))

                if opt_res['success'] == False:
                    #print(f"Failed fit for model: {g_name} (true:{true_graph})")
                    graph_ll += -1e10
                           
                graph_ll += -1*opt_res['fun']
            ll.append(graph_ll)
            graph_complexity = G.sum()
            
            # Note: scipy minimize returns the negative log likelihood value
            bic_score = graph_ll - .5 * np.log(n) * graph_complexity
            bic_scores.append(bic_score)

        # transition to graphs w/ probability proportional to BIC scores
        # option 1: softmax
        p = softmax(bic_scores)
        next_ix = np.random.choice(len(candidate_graphs), p = p)
        return adj_matrices[candidate_graphs[next_ix]]


    def step(self, a):
        assert a >= 0, "action out of range"
        assert a < self.n_actions, "action out of range"
        done = False
        r = 0
        if a == self.n_actions - 1:
            # STOP action
            done = True
            if (self.true_graph == self.curr_graph).all():
                r = 1
        else:
            # experiment action
            node, n_samples = self.experiment_actions[a]
            self.n_experiments += n_samples
            self.curr_graph = self.transition_graph(node, n_samples)
            r = self.correct_edges_frac # shaped reward based on number of correct edges
        self.t += 1
        if self.t >= self.max_time:
            done = True
        return self.create_state, r, done, None


In [31]:
# sanity checks for MDP defined above
labeled_mdp = LabeledMDP("A1")
s = labeled_mdp.reset()
print(s)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]


## Run this cell to experiment with discovering the best graph based on BIC score

In [250]:
from scipy import optimize
from scipy.stats import norm

K=3
sigma = 1.0
candidate_graphs = ["A1","A2","A3","A4","A5"]
for i,true_graph in enumerate(candidate_graphs):    
    for n in range(5,30,5):
        x,Y = generate_dataset(n,true_graph,sigma)
        ll=[]
        bic_scores = []
        for g_name in candidate_graphs:
            G = np.array(adj_matrices[g_name])
            graph_ll=0
            for var in range(K):
                data = Y[var]
                predictors = np.einsum('ij,ij->ij',np.tile(G[var],(n,1)),x)
                opt_res = optimize.minimize(fun = log_lik, 
                                        x0 = [0, 0, 0], 
                                        args = (data, predictors))

                if opt_res['success'] == False:
                    #print(f"Failed fit for model: {g_name} (true:{true_graph})")
                    graph_ll += -1e10
                           
                graph_ll += -1*opt_res['fun']
            ll.append(graph_ll)
            graph_complexity = G.sum()
            
            # Note: scipy minimize returns the negative log likelihood value
            bic_score = graph_ll - .5*np.log(N)*graph_complexity
            bic_scores.append(bic_score)                    

        best_graph_ind = np.argmax(bic_scores)
        print(f"For N={n}, the best graph/BIC: {candidate_graphs[best_graph_ind]}/{np.round(bic_scores[best_graph_ind],2)}, true graph: {true_graph}/{np.round(bic_scores[i],2)}")

For N=5, the best graph/BIC: A1/-20.62, true graph: A1/-20.62
For N=10, the best graph/BIC: A1/-41.33, true graph: A1/-41.33
For N=15, the best graph/BIC: A1/-62.17, true graph: A1/-62.17
For N=20, the best graph/BIC: A1/-92.31, true graph: A1/-92.31
For N=25, the best graph/BIC: A1/-112.32, true graph: A1/-112.32
For N=5, the best graph/BIC: A2/-25.82, true graph: A2/-25.82
For N=10, the best graph/BIC: A2/-43.79, true graph: A2/-43.79
For N=15, the best graph/BIC: A2/-64.29, true graph: A2/-64.29
For N=20, the best graph/BIC: A2/-82.21, true graph: A2/-82.21
For N=25, the best graph/BIC: A2/-113.44, true graph: A2/-113.44
For N=5, the best graph/BIC: A2/-22.9, true graph: A3/-22.99
For N=10, the best graph/BIC: A3/-47.02, true graph: A3/-47.02
For N=15, the best graph/BIC: A3/-68.84, true graph: A3/-68.84
For N=20, the best graph/BIC: A3/-90.13, true graph: A3/-90.13
For N=25, the best graph/BIC: A3/-120.81, true graph: A3/-120.81
For N=5, the best graph/BIC: A4/-21.4, true graph: A4